In [ ]:
import pandas as pd
import numpy as np
from gensim import corpora
from tqdm import tqdm

In [ ]:
# Read Data (generated by data_preprocessing.ipynb)
groups = pd.read_json('CrossValidation_5%9_group1_Leave1Out+99NegSamples.json')
print('num of users: ', len(groups))

In [ ]:
# Fit Model and Make Recommendations
K = *** decided by the user ***
alpha = *** decided by the user ***
beta = *** decided by the user ***
gamma = *** decided by the user ***
iter_times = *** decided by the user ***
num_recommend = *** decided by the user ***

In [ ]:
# Five Folds Cross Validation
def evaulate_topN_negative_samples(N, recommendation):
    match_people = 0
    NDCG = 0
    pop = 0
    for user in range(len(recommendation)):
        i_truth = 0
        u_recommend = recommendation[user][:N]
        if i_truth in u_recommend:
            match_people += 1
            i_truth_rank = u_recommend.index(i_truth)
            NDCG += 1 / np.log2(i_truth_rank + 2)
        else:
            pass
        first_item = u_recommend[0]
        pop += dictionary.dfs[first_item]
    HR = match_people / len(recommendation)
    NDCG = NDCG / len(recommendation)
    POP = pop / len(recommendation)
    return HR, NDCG, POP

TopN_global = np.array([0]*num_recommend)
N_items_global = np.array([0]*num_recommend)
HRs_global = np.array([0]*num_recommend)
NDCGs_global = np.array([0]*num_recommend)

for fold in range(1, 6):
    # process
    str_Train = [str(i)[1:-1].split(', ') for i in groups['Fold' + str(fold) + '_train']]
    str_Test = [str(i)[1:-1].split(', ') for i in groups['Fold' + str(fold) + '_test']]
    print(' //// Dictionary')
    dictionary = corpora.Dictionary(str_Train)
    print(' //// num of items:', len(dictionary))
    print(' //// dictionary.doc2idx')
    pro_Train = [dictionary.doc2idx(i) for i in str_Train]
    pro_Test = [dictionary.doc2idx(i) for i in str_Test]
    print(' //// Use pro_Train as INPUT and pro_Test as OUTPUT')
    
    PELDA = PELDA(numTopics=K, alpha, beta, gamma)
    PELDA.fit(pro_Train, iter_times)
    PELDA.params_estimate()
    PELDA.negative_recommend(pro_Test, num_recommend)
    recommend = PELDA.read_negative_recommend()

    # print evaluation results for each fold
    TopN = []
    N_items = []
    HRs = []
    NDCGs = []
    for i in tqdm(range(1, num_recommend+1)):
        HR, NDCG, POP = evaulate_topN_negative_samples(i, recommend)
        TopN.append(i)
        HRs.append(HR)
        NDCGs.append(NDCG)
    results = pd.DataFrame({'TopN': TopN, 'ConversionRate': HRs, 'NDCG': NDCGs})
    print(results)
    results.to_csv('PELDA_results_'+str(fold)+'.csv', index=False)
    
    TopN_global += np.array(TopN)
    N_items_global += np.array(N_items)
    HRs_global += np.array(HRs)
    NDCGs_global += np.array(NDCGs)

In [ ]:
# Print Evaluation Results for Five Folds
print('////////////   Negative Sample Evaluation  ////////////')
results = pd.DataFrame({'TopN': TopN_global/5, 'ConversionRate': HRs_global/5, 'NDCG': NDCGs_global/5})
print(results)
results.to_csv('PELDA_results.csv', index=False)